### Step 1: Load Raw HTML Pages
*Load all webpages from your file system into Python as text in a list format*

In [1]:
# Find current working directory and its sub-directories

import os

cwd = os.getcwd()
dirs = os.listdir()

print(cwd)
print(dirs)

/Users/samanthaberk/Documents/resume-job-posting-nlp-project
['.DS_Store', 'LICENSE', 'environment.yml', 'Extracting Text from Online Job Postings.ipynb', 'resume_project.pickle', 'README.md', '.gitignore', '.ipynb_checkpoints', '.git', 'data']


In [2]:
# Build a path to the HTML job postings

path = os.path.join(cwd, dirs[dirs.index('data')])
print(path)

files = os.listdir(path)
path = os.path.join(path, files[files.index('html_job_postings')])
print(path)

/Users/samanthaberk/Documents/resume-job-posting-nlp-project/data
/Users/samanthaberk/Documents/resume-job-posting-nlp-project/data/html_job_postings


In [3]:
# Iterate through all files in the path and store them in a list

dups = set()
job_postings = []

for posting in os.listdir(path):
    if posting not in dups:
        job_postings.append(posting)
        dups.add(posting)

### Step 2: Prepare to store extracted data
*Create a pandas Dataframe with sensible column names. Also create helper function that will store the webpage sections in the dataframe*

In [4]:
import pandas as pd

# Helper function to create dataframe to store jobs and requirements
def create_df(data):
    header = 'requirement'
    result = ['Title']
    
    i = 1
    while i < len(data):
        result.append(header + str(i))
        i += 1
    df = pd.DataFrame(data, result)
    return df


### Step 3: Extract relevant sections from each job posting

*Determine which sections should be extracted and stored. Where do most of the required skills end up showing up? Which HTML tags typically contain the skills requirements for jobs?*

In [8]:
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen

# Iterate through each posting and extract sections to store in a data frame
frames = []
for posting in job_postings:
    url = os.path.join('file://' + path, posting)
    html_contents = urlopen(url).read()
    soup = bs(html_contents)
    title = [soup.title.text]
    body = soup.body
    requirements = [bullet.text for bullet in body.find_all('li')]
    data = title + requirements
    df = create_df(data)
    frames.append(df)

final_df = pd.concat(frames)
print(final_df)

                                             0
Title  Quantitative Analyst - Boston, MA 02116
                                                               0
Title                         Data Scientist - Mountain View, CA
requirement1   \nHelp senior members of the team to explore, ...
requirement2   \nUse Hadoop, Spark and Amazon Athena on a dai...
requirement3   \nDive deep into our rich set of location data...
requirement4   \nCollaborate with peer data scientists, engin...
requirement5   \nMaster degree in Computer Science, Statistic...
requirement6   \nHas experience in Statistics, Machine Learning.
requirement7                                \nFluency in Python.
requirement8   \nSignificant experience with SQL; some relati...
requirement9   \nFamiliarity with open source machine learnin...
requirement10  \nHas experience in Amazon Web Services is a p...
requirement11                  \nExcellent communication skills.
                                                             

                                                               0
Title                       Data Scientist - Grapevine, TX 76051
requirement1                                     Health benefits
requirement2                                       401(k)Program
requirement3               Daily dress code of “business casual”
requirement4                         A positive work environment
requirement5                  \nData Science: 2 years (Required)
requirement6                 \nData Storage: 2 years (Preferred)
requirement7           \nPredictive Modeling: 2 years (Required)
requirement8                                      \nOne location
requirement9                                  \nHealth insurance
requirement10                                 \nDental insurance
requirement11                                 \nVision insurance
requirement12                                  \nRetirement plan
requirement13                                    \nPaid time off
requirement14            

                                                              0
Title              Test Automation Engineer - San Francisco, CA
requirement1  Hands-on experience in development and test la...
requirement2  Storage testing experience, especially with NA...
requirement3  Experience configuring, troubleshooting and op...
requirement4  Ability to write and troubleshoot Perl scripts...
requirement5  Ability to conduct disciplined automation test...
requirement6                    File system testing experience.
requirement7                    Performance testing experience.
requirement8    Other operating systems: Solaris, Windows 2000.
requirement9             L2 and L3 networking (routing, VLANs).
                                                               0
Title          Software Developer – Machine-learning/Artifici...
requirement1                                               RADAR
requirement2   Signal processing, system design, operational ...
requirement3                        

                                                               0
Title                        Data Scientist - New York, NY 10011
requirement1   Conduct research on some of the world’s most i...
requirement2   Plan, develop, and apply cutting-edge machine ...
requirement3   Write production-ready code to analyze, struct...
requirement4   Design systems to monitor the results of model...
requirement5   Lead projects from start to finish, collaborat...
requirement6   At least 5 years of relevant professional expe...
requirement7   Strong knowledge of machine learning, computer...
requirement8   Strong programming skills in Python, R, and/or...
requirement9   Experience with NumPy, SciPy, Pandas, Scikit-L...
requirement10  Experience with building distributed machine l...
requirement11  Experience with several of the following conce...
requirement12  Publications in communities such as NIPS, ICML...
requirement13  GitHub projects demonstrating your creative dr...
requirement14  Kaggle win

                                                               0
Title                        Data BI Analyst - Andover, MA 01810
requirement1    Collaborate with leaders of functional areas ...
requirement2    Comprehend, extract, and massage information ...
requirement3    Create informative, actionable, and repeatabl...
requirement4    Perform deep-dive analysis, including the app...
requirement5    Develop data visualizations, including report...
requirement6    Must be proficient with some powerful program...
requirement7    Must have basic knowledge of statistics (norm...
requirement8    Must be willing and eager to provide live pre...
requirement9    Must have experience with some variety of reg...
requirement10  Hands on experience conducting quantitative an...
requirement11  Coursework or experience with Excel (pivot tab...
requirement12  Analytical, creative, and innovative approach ...
requirement13     Strong written and verbal communication skill.
requirement14  Management

...

                                                              0
Title                               Data engineer - Fremont, CA
requirement1        Web development experience (AngularJS, D3).
requirement2  Experience in a statistical programming langua...
requirement3  Experience in at least one data visualization ...
requirement4   Building and scaling Machine Learning frameworks
requirement5                        Hadoop (Hive, Spark, UDF's)
requirement6        Web development experience (AngularJS, D3).
requirement7  5+ years of relevant experience in one of the ...
requirement8  5+ years of hands-on experience in writing com...
requirement9  2+ years of experience in scripting languages ...
                                                               0
Title          IED and Insurgent Network Analyst - Journeyman...
requirement1   Produces intelligence assessments for formal p...
requirement2   Conducts all-source intelligence analysis to i...
requirement3   Produce targeting sup

                                                              0
Title         Data Science & Analytics Intern (Paid) - Summe...
requirement1  \nWork directly with a mentor in your field of...
requirement2  \nExperience our products up close by touring ...
requirement3  \nInteract directly with our company leaders t...
requirement4  \nBuild a lasting professional network through...
requirement5  \nGain the skills and knowledge to become a fu...
                                                               0
Title                             Data Scientist - Sunnyvale, CA
requirement1   Responsible for one or more ML related applica...
requirement2   Develops and demonstrates viability of novel s...
requirement3   Be a part of an engineering team with total re...
requirement4   PhD program in Computer Science, Mathematics o...
requirement5   Academic background in deep learning, natural ...
requirement6   5+ years of programming experience, proficienc...
requirement7   Strong knowledge 

                                                               0
Title          Data Scientist Intern, PhD, 2020 - Los Angeles...
requirement1   \nUpdated resume - please include the relevant...
requirement2   \nTranscript - unofficial or official; must be...
requirement3   \nCurrently pursuing a PhD in a quantitative d...
requirement4   Experience using technology to work with datas...
requirement5   Experience with statistical data analysis such...
requirement6   \nStrong track record of developing intellectu...
requirement7             Strong analytical and technical skills.
requirement8   Returning to a degree program after the intern...
requirement9         Authorization to work in the United States.
requirement10  Work with large, complex data sets. Solve diff...
requirement11  Interact cross-functionally with a wide variet...
requirement12  Provide research on topics including Google's ...
requirement13  Produce quantitative and qualitative modeling ...
                         

                                                               0
Title          New College Grad - Cybersecurity (Masters Degr...
requirement1   \nAdvanced Degree (e.g. Masters, MBA, JD, MD, ...
requirement2   \nMinimum GPA of 3.0 out of 4.0 (or equivalent...
requirement3   \nMasters degree in Cybersecurity, CS, Enginee...
requirement4   \nSupporting software development / QA special...
requirement5   \nTech skills: Burp Suite, IDA pro, APKTool, H...
requirement6                     \nSOAP / REST API, Web Services
requirement7   \nDevelopment tools such as Maven, Jenkins and...
requirement8   \nDifferent programming and scripting language...
requirement9   \nSoftware Development, Programming, and debug...
requirement10  \nApplying Machine Learning/Artificial Intelli...
requirement11          \nStatistical and machine learning models
requirement12  \nExceptional coding skills and experience wit...
requirement13  \nExperience with tools for statistical comput...
requirement14  \nIdentity

ValueError: All objects passed were None

### Step 4: Save Dataframe for later

In [ ]:
DATA = "resume_project.pickle"
final_df.to_pickle(DATA)

In [9]:
read_df = pd.read_pickle(DATA)
read_df.head()

,0
Title,"Quantitative Analyst - Boston, MA 02116"
Title,"Data Scientist - Mountain View, CA"
requirement1,"\nHelp senior members of the team to explore, ..."
requirement2,"\nUse Hadoop, Spark and Amazon Athena on a dai..."
requirement3,\nDive deep into our rich set of location data...
